In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("MainDataSet15m.csv").tail(1000)

In [14]:
df['Prev_Candle_Signal'] =df['Candle_Signal'].shift(1)
df['Prev_Candle'] = df['Candle'].shift(1)


In [15]:
conditions = [
    (df["Day_Prev_Candle_Signal"] == "Bearish") & (df["SuperTrend"] == -1) & (df["Prev_Candle_Signal"] == "Bearish") ,
     (df["Day_Prev_Candle_Signal"] == "Bullish") & (df["SuperTrend"] == 1) & (df["Prev_Candle_Signal"] == "Bullish"),
]

choices = ["PE", "CE"]

df["TradSide"] = np.select(conditions, choices, default="None")


In [20]:
ce_condition = ((df['High'].shift(1) < df['15EMA'].shift(1)) & (df["Close"] > df['15EMA'])) | ((df['Candle'].shift(2) == "Green") & (df['Candle'].shift(1) == "Red") & (df["Close"] >= df['High'].shift(1)) & (df['Low'].shift(2) <= df['Low'].shift(1)))
pe_condition = ((df['Low'].shift(1) > df['15EMA'].shift(1)) & (df["Close"] < df['15EMA']) )| ((df['Candle'].shift(2) == "Red") & (df['Candle'].shift(1) == "Green") & (df["Close"] <= df['Low'].shift(1)) & (df['High'].shift(2) >= df['High'].shift(1)))
df['EMA_Cross'] = np.select([ce_condition, pe_condition], ['CE', 'PE'], default='NONE')

In [21]:
time_enter = "2024-05-10 09:15"
df[df["Datetime"].str.contains(time_enter)].iloc[-1]

Day_Open                             21990.95
Day_High                              22131.3
Day_Low                               21950.3
Day_Close                             22055.2
Day_SuperTrend                              1
Day_Candle                              Green
Day_CandleBody                          181.0
Day_5EMA                         22183.562039
Day_15EMA                        22323.780063
Day_20EMA                        22334.217971
Day_50EMA                        22224.859767
Day_200EMA                       21002.800289
Day_RSI                             31.210935
Day_upper_shadow_pr                 65.182013
Day_lower_shadow_pr                 34.817987
Day_Candle_Signal                     Neutral
Day_Prev_Candle_Signal                Bearish
Day_Prev_Candle                           Red
Day_Date                           2024-05-10
Day_SwingMin                          21137.2
Day_SwingMax                          22794.7
Datetime                  2024-05-